In [1]:
'''
    This Notebook is for learning only
'''

In [2]:
# Import
import tensorflow as tf
import numpy as np
import pandas as pd
from collections import deque
import random

import tetris as tetris


In [3]:

# Hyperparameter 

num_episodes = 500
num_exploration_episodes = 100
max_len_episode = 1000
batch_size = 40
learning_rate = 0.005
gamma = 0.95
initial_epsilon = 1.0
final_epsilon = 0.01

eps_decay = 0.995
eps_min = 0.01

In [ ]:

class QNetwork(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.state_dim  = 216
        self.action_dim = 36
        self.epsilon = 1.
        self.dense1 = tf.keras.layers.Dense(units=216, input_dim=216,activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=116, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(units=36, activation=tf.nn.relu)
        self.dense4 = tf.keras.layers.Dense(units=self.action_dim)
        
        
        
        self.model = self.create_model()
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
    
    def create_model(self):
#         model = tf.keras.Sequential([
#             Input((self.state_dim,)),
#             Dense(32, activation='relu'),
#             Dense(16, activation='relu'),
#             Dense(self.action_dim)
#         ])
        model = tf.keras.models.Sequential()
        model.add(self.dense1)
        model.add(self.dense2)
        model.add(self.dense3)
        model.add(self.dense4)
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate))
        return model
    
    def predict(self, state):
        return self.model.predict(state)
    
    def get_action(self, state):
        state = np.reshape(state, [1, self.state_dim])
        self.epsilon *= eps_decay
        self.epsilon = max(self.epsilon, eps_min)
        q_value = self.predict(state)[0]
        if np.random.random() < self.epsilon:
            return random.randint(0, 35)
        
        
        return np.argmax(q_value)
    
    def train(self, states, targets):
        self.model.fit(states, targets, epochs=1, verbose=0)
class ReplayBuffer:
    def __init__(self, capacity=100000):
        self.buffer = deque(maxlen=capacity)
    
    def put(self, state, action, reward, next_state, done):
        self.buffer.append([state, action, reward, next_state, done])
    
    def sample(self):
        sample = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, done = map(np.asarray, zip(*sample))
        states = np.array(states).reshape(batch_size, -1)
        next_states = np.array(next_states).reshape(batch_size, -1)
        return states, actions, rewards, next_states, done
    
    def size(self):
        return len(self.buffer)
    
class Agent:
    def __init__(self, env):
        self.env = env
        self.state_dim = 216
        self.action_dim = 36                                 # move 0 - 8 rotate 0 - 3 = 9*4

        self.model = QNetwork()
        self.target_model = QNetwork()
        self.target_update()

        self.buffer = ReplayBuffer()
        
        self.max_score_now = 0
    def target_update(self):
        weights = self.model.model.get_weights()
        self.target_model.model.set_weights(weights)
    
    def replay(self):
        for _ in range(10):
            states, actions, rewards, next_states, done = self.buffer.sample()
            targets = self.target_model.predict(states)
            next_q_values = self.target_model.predict(next_states).max(axis=1)
            #print(states, actions, rewards, next_states, done,next_q_values )
            targets[range(batch_size), actions] = rewards + (1-done) * next_q_values * gamma
            self.model.train(states, targets)
    
    def train(self, max_episodes=1000):
        print("Start.")
        for ep in range(max_episodes):
            done, total_reward = False, 0
            state = self.env.reset()
            while not done:
                action = self.model.get_action(state)
                #print(type(action),action)
                next_state, reward, done, info = self.env.step_action(int(action))
                self.buffer.put(state, action, reward, next_state, done)
                total_reward += reward
                state = next_state
            
            
            
            if self.buffer.size() >= batch_size:
                self.replay()
            self.target_update()
            if self.env.score > self.max_score_now:
                self.max_score_now = self.env.score
                for i in range(len(self.env.pannel)):
                    print(self.env.pannel[i])
            print("Total Steps",self.env.total_steps,"Score",self.env.score,"Max Score",self.max_score_now)
            print('EP{} EpisodeReward={}'.format(ep, total_reward))
            print("===========================================================")
            #wandb.log({'Reward': total_reward})


def main():
    
    env = tetris.Tertris(10,20)
    agent = Agent(env)
    agent.train(max_episodes=100000)

if __name__ == "__main__":
    main()

Start.
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
[0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
[1, 1, 1, 1, 0, 0, 1, 1, 0, 0]
[1, 1, 1, 1, 0, 0, 1, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 0, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 1, 1, 0, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 0, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 1, 1, 1, 1, 0]
[0, 0, 1, 1, 0, 1, 1, 1, 1, 0]
[0, 1, 1, 0, 1, 1, 0, 0, 1, 1]
[0, 1, 1, 0, 1, 1, 0, 0, 1, 1]
Total Steps 26 Score 26 Max Score 26
EP0 EpisodeReward=15
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0

Total Steps 13 Score 13 Max Score 231
EP59 EpisodeReward=2
Total Steps 22 Score 22 Max Score 231
EP60 EpisodeReward=11
Total Steps 18 Score 18 Max Score 231
EP61 EpisodeReward=7
Total Steps 32 Score 32 Max Score 231
EP62 EpisodeReward=21
Total Steps 23 Score 23 Max Score 231
EP63 EpisodeReward=12
Total Steps 32 Score 32 Max Score 231
EP64 EpisodeReward=21
Total Steps 35 Score 35 Max Score 231
EP65 EpisodeReward=24
Total Steps 32 Score 32 Max Score 231
EP66 EpisodeReward=21
Total Steps 27 Score 27 Max Score 231
EP67 EpisodeReward=16
Total Steps 33 Score 33 Max Score 231
EP68 EpisodeReward=22
Total Steps 31 Score 31 Max Score 231
EP69 EpisodeReward=20
Total Steps 28 Score 28 Max Score 231
EP70 EpisodeReward=17
Total Steps 33 Score 33 Max Score 231
EP71 EpisodeReward=22
Total Steps 27 Score 27 Max Score 231
EP72 EpisodeReward=16
Total Steps 26 Score 26 Max Score 231
EP73 EpisodeReward=15
Total Steps 19 Score 19 Max Score 231
EP74 EpisodeReward=8
Total Steps 26 Score 26 Max Score 231
EP75 

Total Steps 32 Score 32 Max Score 651
EP122 EpisodeReward=21
Total Steps 31 Score 31 Max Score 651
EP123 EpisodeReward=20
Total Steps 31 Score 31 Max Score 651
EP124 EpisodeReward=20
Total Steps 33 Score 33 Max Score 651
EP125 EpisodeReward=22
Total Steps 33 Score 33 Max Score 651
EP126 EpisodeReward=22
Total Steps 37 Score 37 Max Score 651
EP127 EpisodeReward=26
Total Steps 32 Score 32 Max Score 651
EP128 EpisodeReward=21
Total Steps 32 Score 32 Max Score 651
EP129 EpisodeReward=21
Total Steps 34 Score 34 Max Score 651
EP130 EpisodeReward=23
Total Steps 36 Score 36 Max Score 651
EP131 EpisodeReward=25
Total Steps 30 Score 30 Max Score 651
EP132 EpisodeReward=19
Total Steps 30 Score 30 Max Score 651
EP133 EpisodeReward=19
Total Steps 34 Score 34 Max Score 651
EP134 EpisodeReward=23
Total Steps 38 Score 38 Max Score 651
EP135 EpisodeReward=27
Total Steps 37 Score 37 Max Score 651
EP136 EpisodeReward=26
Total Steps 31 Score 31 Max Score 651
EP137 EpisodeReward=20
Total Steps 29 Score 29 

Total Steps 28 Score 28 Max Score 839
EP185 EpisodeReward=17
Total Steps 35 Score 35 Max Score 839
EP186 EpisodeReward=24
Total Steps 38 Score 238 Max Score 839
EP187 EpisodeReward=227
Total Steps 44 Score 644 Max Score 839
EP188 EpisodeReward=633
Total Steps 33 Score 33 Max Score 839
EP189 EpisodeReward=22
Total Steps 33 Score 33 Max Score 839
EP190 EpisodeReward=22
Total Steps 28 Score 28 Max Score 839
EP191 EpisodeReward=17
Total Steps 34 Score 34 Max Score 839
EP192 EpisodeReward=23
Total Steps 34 Score 34 Max Score 839
EP193 EpisodeReward=23
Total Steps 34 Score 34 Max Score 839
EP194 EpisodeReward=23
Total Steps 32 Score 32 Max Score 839
EP195 EpisodeReward=21
Total Steps 37 Score 237 Max Score 839
EP196 EpisodeReward=226
Total Steps 43 Score 443 Max Score 839
EP197 EpisodeReward=432
[0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 

Total Steps 38 Score 438 Max Score 852
EP247 EpisodeReward=427
Total Steps 39 Score 439 Max Score 852
EP248 EpisodeReward=428
Total Steps 47 Score 447 Max Score 852
EP249 EpisodeReward=436
Total Steps 42 Score 442 Max Score 852
EP250 EpisodeReward=431
Total Steps 36 Score 436 Max Score 852
EP251 EpisodeReward=425
Total Steps 35 Score 435 Max Score 852
EP252 EpisodeReward=424
Total Steps 36 Score 436 Max Score 852
EP253 EpisodeReward=425
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 0, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
[

...

Total Steps 28 Score 428 Max Score 1469
EP359 EpisodeReward=417
Total Steps 28 Score 428 Max Score 1469
EP360 EpisodeReward=417
Total Steps 28 Score 428 Max Score 1469
EP361 EpisodeReward=417
Total Steps 37 Score 837 Max Score 1469
EP362 EpisodeReward=826
Total Steps 53 Score 853 Max Score 1469
EP363 EpisodeReward=842
Total Steps 46 Score 446 Max Score 1469
EP364 EpisodeReward=435
Total Steps 47 Score 447 Max Score 1469
EP365 EpisodeReward=436
Total Steps 46 Score 446 Max Score 1469
EP366 EpisodeReward=435
Total Steps 33 Score 633 Max Score 1469
EP367 EpisodeReward=622
Total Steps 58 Score 858 Max Score 1469
EP368 EpisodeReward=847
Total Steps 40 Score 640 Max Score 1469
EP369 EpisodeReward=629
Total Steps 36 Score 236 Max Score 1469
EP370 EpisodeReward=225
Total Steps 49 Score 449 Max Score 1469
EP371 EpisodeReward=438
Total Steps 44 Score 844 Max Score 1469
EP372 EpisodeReward=833
Total Steps 50 Score 650 Max Score 1469
EP373 EpisodeReward=639
Total Steps 45 Score 845 Max Score 1469


Total Steps 68 Score 1468 Max Score 1976
EP419 EpisodeReward=1457
Total Steps 69 Score 1469 Max Score 1976
EP420 EpisodeReward=1458
Total Steps 50 Score 850 Max Score 1976
EP421 EpisodeReward=839
Total Steps 47 Score 647 Max Score 1976
EP422 EpisodeReward=636
Total Steps 48 Score 648 Max Score 1976
EP423 EpisodeReward=637
Total Steps 46 Score 846 Max Score 1976
EP424 EpisodeReward=835
Total Steps 61 Score 1061 Max Score 1976
EP425 EpisodeReward=1050
Total Steps 62 Score 1462 Max Score 1976
EP426 EpisodeReward=1451
Total Steps 62 Score 1462 Max Score 1976
EP427 EpisodeReward=1451
Total Steps 34 Score 634 Max Score 1976
EP428 EpisodeReward=623
Total Steps 65 Score 1065 Max Score 1976
EP429 EpisodeReward=1054
Total Steps 58 Score 858 Max Score 1976
EP430 EpisodeReward=847
Total Steps 63 Score 1463 Max Score 1976
EP431 EpisodeReward=1452
Total Steps 68 Score 1468 Max Score 1976
EP432 EpisodeReward=1457
Total Steps 63 Score 1063 Max Score 1976
EP433 EpisodeReward=1052
Total Steps 66 Score 1

Total Steps 48 Score 548 Max Score 1976
EP485 EpisodeReward=537
Total Steps 63 Score 1463 Max Score 1976
EP486 EpisodeReward=1452
Total Steps 31 Score 231 Max Score 1976
EP487 EpisodeReward=220
Total Steps 47 Score 647 Max Score 1976
EP488 EpisodeReward=636
Total Steps 30 Score 230 Max Score 1976
EP489 EpisodeReward=219
Total Steps 56 Score 1056 Max Score 1976
EP490 EpisodeReward=1045
Total Steps 41 Score 441 Max Score 1976
EP491 EpisodeReward=430
Total Steps 56 Score 656 Max Score 1976
EP492 EpisodeReward=645
Total Steps 21 Score 21 Max Score 1976
EP493 EpisodeReward=10
Total Steps 50 Score 650 Max Score 1976
EP494 EpisodeReward=639
Total Steps 43 Score 243 Max Score 1976
EP495 EpisodeReward=232
Total Steps 34 Score 34 Max Score 1976
EP496 EpisodeReward=23
Total Steps 24 Score 224 Max Score 1976
EP497 EpisodeReward=213
Total Steps 19 Score 19 Max Score 1976
EP498 EpisodeReward=8
Total Steps 20 Score 20 Max Score 1976
EP499 EpisodeReward=9
Total Steps 19 Score 19 Max Score 1976
EP500 E

Total Steps 62 Score 1062 Max Score 1976
EP552 EpisodeReward=1051
Total Steps 62 Score 1062 Max Score 1976
EP553 EpisodeReward=1051
Total Steps 62 Score 1362 Max Score 1976
EP554 EpisodeReward=1351
Total Steps 62 Score 1362 Max Score 1976
EP555 EpisodeReward=1351
Total Steps 18 Score 18 Max Score 1976
EP556 EpisodeReward=7
Total Steps 42 Score 442 Max Score 1976
EP557 EpisodeReward=431
Total Steps 32 Score 32 Max Score 1976
EP558 EpisodeReward=21
Total Steps 40 Score 240 Max Score 1976
EP559 EpisodeReward=229
Total Steps 36 Score 236 Max Score 1976
EP560 EpisodeReward=225
Total Steps 30 Score 30 Max Score 1976
EP561 EpisodeReward=19
Total Steps 24 Score 24 Max Score 1976
EP562 EpisodeReward=13
Total Steps 34 Score 234 Max Score 1976
EP563 EpisodeReward=223
Total Steps 27 Score 27 Max Score 1976
EP564 EpisodeReward=16
Total Steps 37 Score 337 Max Score 1976
EP565 EpisodeReward=326
Total Steps 37 Score 337 Max Score 1976
EP566 EpisodeReward=326
Total Steps 44 Score 244 Max Score 1976
EP5

Total Steps 25 Score 25 Max Score 1976
EP619 EpisodeReward=14
Total Steps 23 Score 23 Max Score 1976
EP620 EpisodeReward=12
Total Steps 25 Score 25 Max Score 1976
EP621 EpisodeReward=14
Total Steps 26 Score 26 Max Score 1976
EP622 EpisodeReward=15
Total Steps 25 Score 25 Max Score 1976
EP623 EpisodeReward=14
Total Steps 18 Score 18 Max Score 1976
EP624 EpisodeReward=7
Total Steps 18 Score 18 Max Score 1976
EP625 EpisodeReward=7
Total Steps 28 Score 28 Max Score 1976
EP626 EpisodeReward=17
Total Steps 43 Score 443 Max Score 1976
EP627 EpisodeReward=432
Total Steps 25 Score 25 Max Score 1976
EP628 EpisodeReward=14
Total Steps 32 Score 32 Max Score 1976
EP629 EpisodeReward=21
Total Steps 29 Score 29 Max Score 1976
EP630 EpisodeReward=18
Total Steps 55 Score 1055 Max Score 1976
EP631 EpisodeReward=1044
Total Steps 32 Score 232 Max Score 1976
EP632 EpisodeReward=221
Total Steps 24 Score 24 Max Score 1976
EP633 EpisodeReward=13
Total Steps 30 Score 30 Max Score 1976
EP634 EpisodeReward=19
To